In [ ]:
def comparePair1(rec1,rec2):
    n1 = set(rec1['name'])
    n2 = set(rec2['name'])
    a1 = set(rec1['address'])
    a2 = set(rec2['address']) 
    dname = nltk.jaccard_distance(n1,n2)
    dadress = nltk.jaccard_distance(a1,a2)
    if dname < 0.2 or dadress < 0.2:
        return True
    return False 
    ''' Gestion de l input, lecture du fichier excel '''

import pandas as pd

DATASET_FILE = "restaurants-without-IDs.xlsx"
SORTED_FILE = "restaurants-sorted.csv"
SORTING_COLUMN = "name"
SHEET_TO_ANALYZE = "Sheet1"
WINDOW_SIZE = 2
DUPLICATE_FILE = "restaurants-duplicates.xlsx"

def sortFile(inputFile,sortingCol,sheetName,outputFile):
    xl = pd.ExcelFile(inputFile)
    df = xl.parse(sheetName)
    df[sortingCol] = df[sortingCol].astype(str)
    df = df.sort_values(sortingCol)
    df.to_csv(outputFile,index=False)
    
sortFile(DATASET_FILE,SORTING_COLUMN,SHEET_TO_ANALYZE,SORTED_FILE)
''' On observe que les données sont dans l ordre mais rentrees plusieurs fois '''
'''Sort par adress ou name sont interessants'''

'Sort par adress ou name sont interessants'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
''' Definition des mesures de similarite/distance'''
import nltk

pd.set_option('display.max_rows', None)

'''En utilisant Jaccard et les champs name et adress
Les champs city et type ne semblent pas tres pertinents car compose d un ou deux mots
City pourraient etre pertinent si les deux city sont un peu similaires et que les deux premiers champ sont aussi similaires'''


def comparePair1(rec1,rec2):
    n1 = set(rec1['name'])
    n2 = set(rec2['name'])
    a1 = set(rec1['address'])
    a2 = set(rec2['address']) 
    dname = nltk.jaccard_distance(n1,n2)
    dadress = nltk.jaccard_distance(a1,a2)
    if dname < 0.2 or dadress < 0.2:
        return True
    return False



'''Gestion de la fenetre, calcul de la similarite selon la fonction definissant si deux records sont similaires
puis transitive closure step'''
def sortedNeighboorProgram(fileSorted,windowSize,sheetName,similarityFunction,outputFile):
    df = pd.read_csv(fileSorted)
    
    df['Similar to'] = ''
    df.info()
    numberOfRows = len(df)+1
    # Creation de la fenetre
    for i in range(numberOfRows-windowSize):
        window = df.iloc[i:i+windowSize]
        window.index.name = 'index'
        #Pour la premiere fenetre, faire toutes les comparaisons deux à deux
        if i == 0:
            for j in range(windowSize):
                for k in range(j+1,windowSize):
                    areSimilar = similarityFunction(window.iloc[j],window.iloc[k])
                    #S'ils sont similaires, ajout de chacun à la liste des similaires de l'autre
                    if areSimilar:
                        window.at[j,'Similar to']  = window.at[j,'Similar to'] + ","  + str(k)
                        window.at[k,'Similar to']  = window.at[k,'Similar to'] + "," + str(j)
        #Sinon faire les comparaisons du nouveau record avec tous les anciens records de la fenetre
        else:
            lastRow = window.iloc[windowSize-1]
            lastRowIndex = window.index[windowSize-1]
            for j in range(i,i+windowSize-1):
                areSimilar = similarityFunction(lastRow,window.loc[j])
                if areSimilar:
                    window.at[j,'Similar to']  = window.at[j,'Similar to'] + "," + str(lastRowIndex)
                    lastRow['Similar to']  = lastRow['Similar to'] + "," + str(j)
    
    #transitive closure step - Passage en set de la colonne Similar to et deletion des rows sans duplicate
    df['Similar to'] = df['Similar to'].str.split(',')
    for i in range(numberOfRows-1):
        df.at[i,'Similar to'] = set(df.at[i,'Similar to'] )
        df.at[i,'Similar to'].remove('')   
    
    df = df.loc[(df['Similar to'].astype(bool)) ]
    
    

    '''Transitive closure step - pour chaque record on va chercher tous les éléments similaires en allant chercher dans 
    ses éléments similaires puis dans les elements similaires de ses elements simalaires...'''
    for i in df.index:
        closure = set(map(int, df.at[i,'Similar to']))
        while True:
            for dup in closure:
                closure_dup = set(map(int, df.at[dup,'Similar to']))
                new_closure = closure | closure_dup
            
            if new_closure == closure:
                break
            
            closure = new_closure
        
        df.at[i,'Similar to'] = closure
        

    #display(df.iloc[0:800])
    
    display(df)  
    df.to_excel(outputFile)
        

        
        
        
sortedNeighboorProgram(SORTED_FILE,WINDOW_SIZE,SHEET_TO_ANALYZE,comparePair1,DUPLICATE_FILE)

    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        865 non-null    object
 1   address     865 non-null    object
 2   city        865 non-null    object
 3   cuisine     865 non-null    object
 4   Similar to  865 non-null    object
dtypes: object(5)
memory usage: 33.9+ KB


,name,address,city,cuisine,Similar to
2,21 club,21 w. 52nd st.,new york city,american (new),"{2, 3}"
3,21 club,21 w. 52nd st.,new york,american,"{2, 3}"
7,abruzzi,2355 peachtree rd. ne,atlanta,italian,"{8, 7}"
8,abruzzi,2355 peachtree rd. peachtree battle shopping...,atlanta,italian,"{8, 7}"
15,alain rondelli,126 clement st.,san francisco,french (new),"{16, 15}"
16,alain rondelli,126 clement st.,san francisco,french,"{16, 15}"
29,aqua,252 california st.,san francisco,seafood,"{29, 30}"
30,aqua,252 california st.,san francisco,american (new),"{29, 30}"
31,aquavit,13 w. 54th st.,new york,continental,"{32, 31}"
32,aquavit,13 w. 54th st.,new york city,scandinavian,"{32, 31}"


In [ ]:
''' Exemple implémenter la fonction comparePair2 '''

''' Autre fonction/regle pour dire si deux records sont similaires sont definissables'''
def comparePair2(rec1,rec2):
    n1 = set(rec1['name'])
    n2 = set(rec2['name'])
    a1 = set(rec1['address'])
    a2 = set(rec2['address']) 
    dname = nltk.jaccard_distance(n1,n2)
    dadress = nltk.jaccard_distance(a1,a2)
    if dname < 0.2 and dadress < 0.2:
        return True
    return False


sortedNeighboorProgram(SORTED_FILE,WINDOW_SIZE,SHEET_TO_ANALYZE,comparePair2,DUPLICATE_FILE)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        865 non-null    object
 1   address     865 non-null    object
 2   city        865 non-null    object
 3   cuisine     865 non-null    object
 4   Similar to  865 non-null    object
dtypes: object(5)
memory usage: 33.9+ KB


,name,address,city,cuisine,Similar to
2,21 club,21 w. 52nd st.,new york city,american (new),"{2, 3}"
3,21 club,21 w. 52nd st.,new york,american,"{2, 3}"
15,alain rondelli,126 clement st.,san francisco,french (new),"{16, 15}"
16,alain rondelli,126 clement st.,san francisco,french,"{16, 15}"
29,aqua,252 california st.,san francisco,seafood,"{29, 30}"
30,aqua,252 california st.,san francisco,american (new),"{29, 30}"
31,aquavit,13 w. 54th st.,new york,continental,"{32, 31}"
32,aquavit,13 w. 54th st.,new york city,scandinavian,"{32, 31}"
35,arnie mortons of chicago,435 s. la cienega blv.,los angeles,american,"{35, 36}"
36,arnie mortons of chicago,435 s. la cienega blvd.,los angeles,steakhouses,"{35, 36}"


In [ ]:
''' Exemple implémenter la fonction comparePair3 '''
sortedNeighboorProgram(SORTED_FILE,WINDOW_SIZE,SHEET_TO_ANALYZE,comparePair3,DUPLICATE_FILE)

In [ ]:
#règles pour trouver les duplicats avec la distance edit_distance au lieu de jaccard_distance
#on a utilisé edit_distance pour comparer les séquences dans l'objectif de trouver des duplicates
#choix de implémenter comparePair 3: '''En utilisant edit_distance et les champs name et adress
#Les champs city et type ne nous a pas semblé car composé de plusieurs mots
#City pourraient etre pertinent si les deux city sont un peu similaires et que les deux premiers champ sont aussi similaires'''
#si on a un nombre très petit d'opérations , alors on a similarité entre les mots

def comparePair3(rec1,rec2):
    n1 = rec1['name']
    n2 = rec2['name']
    a1 = rec1['address']
    a2 = rec2['address'] 
    dname = nltk.edit_distance(n1,n2)
    print(dname)
    dadress = nltk.edit_distance(a1,a2)
    print(dadress)
    if dname < 5 or dadress < 5:
        return True
    return False

sortedNeighboorProgram(SORTED_FILE,WINDOW_SIZE,SHEET_TO_ANALYZE,comparePair3,DUPLICATE_FILE)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        865 non-null    object
 1   address     865 non-null    object
 2   city        865 non-null    object
 3   cuisine     865 non-null    object
 4   Similar to  865 non-null    object
dtypes: object(5)
memory usage: 33.9+ KB
5
29
5
30
0
0
6
10
14
9
13
16
5
17
0
31
8
42
15
20
12
22
15
17
14
31
8
30
12
18
0
0
10
10
19
24
17
24
16
11
15
26
12
23
12
32
19
33
16
13
13
35
3
39
12
14
12
17
0
0
3
13
0
0
4
7
4
9
18
18
0
1
18
14
8
0
13
30
9
30
13
14
15
34
9
34
0
0
9
29
0
20
9
10
14
12
13
18
12
16
9
35
11
33
10
15
5
11
12
15
20
24
22
25
17
16
15
19
9
19
11
24
10
18
7
19
10
13
6
26
0
24
5
18
6
32
8
30
7
31
16
32
15
11
3
17
5
17
4
16
6
11
5
11
9
9
3
7
26
7
24
18
8
16
3
14
11
5
12
33
9
37
8
9
0
0
5
22
16
23
14
48
0
39
12
15
15
16
18
18
19
18
9
17
11
14
12
24
14
22
1
15
41
31
37
32
13
17
13
19
0
3
6
17
1

,name,address,city,cuisine,Similar to
2,21 club,21 w. 52nd st.,new york city,american (new),"{2, 3}"
3,21 club,21 w. 52nd st.,new york,american,"{2, 3}"
7,abruzzi,2355 peachtree rd. ne,atlanta,italian,"{8, 7}"
8,abruzzi,2355 peachtree rd. peachtree battle shopping...,atlanta,italian,"{8, 7}"
15,alain rondelli,126 clement st.,san francisco,french (new),"{16, 15}"
16,alain rondelli,126 clement st.,san francisco,french,"{16, 15}"
29,aqua,252 california st.,san francisco,seafood,"{29, 30}"
30,aqua,252 california st.,san francisco,american (new),"{29, 30}"
31,aquavit,13 w. 54th st.,new york,continental,"{32, 31}"
32,aquavit,13 w. 54th st.,new york city,scandinavian,"{32, 31}"


In [ ]:
#module difflib fournit des classes et des fonctions pour comparer les séquences et pour détecter des duplicates 
#Il peut être utilisé, par exemple, pour comparer des fichiers et peut produire des informations 
#sur les différences dans divers formats, y compris HTML et les différences contextuelles et unifiées. 
#on a utilisé difflib.SequenceMatcher et ratio() pour comparer les séquences dans l'objectif de trouver des duplicates
import difflib
#choix de implémenter comparePair 4: '''En utilisant la librairie difflib et les champs name et adress
#Les champs city et type ne nous a pas semblé car composé de plusieurs mots
#City pourraient etre pertinent si les deux city sont un peu similaires et que les deux premiers champ sont aussi similaires'''
#si on a un ratio plus grand que 0.6,alors on a similarité entre les mots
def comparePair4(rec1,rec2):
    n1 = rec1['name']
    n2 = rec2['name']
    a1 = rec1['address']
    a2 = rec2['address'] 
    
    s1 = difflib.SequenceMatcher(lambda x: x == " ",n1,n2)
    s2 = difflib.SequenceMatcher(lambda x: x == " ",a1,a2)
    
    #affichage du ratio 
    print(round(s1.ratio(),2))
    print(round(s2.ratio(),2))
    
    if s1.ratio()>0.6 or s2.ratio()>0.6:
        return True
    return False
sortedNeighboorProgram(SORTED_FILE,WINDOW_SIZE,SHEET_TO_ANALYZE,comparePair4,DUPLICATE_FILE)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        865 non-null    object
 1   address     865 non-null    object
 2   city        865 non-null    object
 3   cuisine     865 non-null    object
 4   Similar to  865 non-null    object
dtypes: object(5)
memory usage: 33.9+ KB
0.27
0.31
0.14
0.21
1.0
1.0
0.18
0.45
0.0
0.43
0.11
0.29
0.33
0.36
1.0
0.56
0.24
0.25
0.21
0.22
0.37
0.16
0.23
0.45
0.22
0.32
0.17
0.36
0.24
0.34
1.0
1.0
0.42
0.44
0.22
0.21
0.44
0.21
0.2
0.35
0.17
0.26
0.33
0.31
0.31
0.3
0.28
0.31
0.38
0.25
0.38
0.28
0.75
0.22
0.27
0.49
0.22
0.21
1.0
1.0
0.73
0.35
1.0
1.0
0.43
0.53
0.57
0.63
0.39
0.19
1.0
0.98
0.36
0.5
0.69
1.0
0.33
0.11
0.22
0.18
0.2
0.29
0.33
0.15
0.33
0.25
1.0
1.0
0.22
0.23
1.0
0.64
0.29
0.53
0.3
0.36
0.34
0.4
0.39
0.39
0.36
0.17
0.32
0.3
0.5
0.17
0.47
0.42
0.38
0.27
0.19
0.31
0.17
0.25
0.2
0.45
0.24
0.27
0.3
0.38


,name,address,city,cuisine,Similar to
2,21 club,21 w. 52nd st.,new york city,american (new),"{2, 3}"
3,21 club,21 w. 52nd st.,new york,american,"{2, 3}"
7,abruzzi,2355 peachtree rd. ne,atlanta,italian,"{8, 7}"
8,abruzzi,2355 peachtree rd. peachtree battle shopping...,atlanta,italian,"{8, 7}"
15,alain rondelli,126 clement st.,san francisco,french (new),"{16, 15}"
16,alain rondelli,126 clement st.,san francisco,french,"{16, 15}"
26,anthonys,3109 piedmont rd. just south of peachtree rd.,atlanta,american,"{26, 27}"
27,antonios,3700 w. flamingo,las vegas,italian,"{26, 27}"
29,aqua,252 california st.,san francisco,seafood,"{32, 29, 30, 31}"
30,aqua,252 california st.,san francisco,american (new),"{32, 29, 30, 31}"
